In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Function to preprocess images
def preprocess_images(img):
    img = cv2.resize(img, (224, 224))  # Resize image to fixed size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
    img = img / 255.0  # Normalize pixel values to [0, 1] range
    img = img.flatten()  # Flatten image into a 1D array
    return img

# Set up data and labels arrays
data = []
labels = []

# Path to the COVID-19 infected X-ray images
covid_folder_path = "/content/drive/MyDrive/4th sem/JUPYTERFILES/ML project/COVID19-xray"
for filename in os.listdir(covid_folder_path):
    img_path = os.path.join(covid_folder_path, filename)
    img = cv2.imread(img_path)
    img = preprocess_images(img)
    data.append(img)
    labels.append(1)  # 1 represents COVID-19 infected

# Path to the COVID-free (Normal) X-ray images
normal_folder_path = "/content/drive/MyDrive/4th sem/JUPYTERFILES/ML project/NORMALXray"
for filename in os.listdir(normal_folder_path):
    img_path = os.path.join(normal_folder_path, filename)
    img = cv2.imread(img_path)
    img = preprocess_images(img)
    data.append(img)
    labels.append(0)  # 0 represents COVID-free

# Convert data and labels arrays to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Define the ensemble model with Gradient Boosting
ensemble_model = GradientBoostingClassifier(n_estimators=10)

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Evaluate the ensemble model
accuracy = ensemble_model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9598765432098766


Mounted at /content/drive


In [ ]:
# Create a pipeline
pipeline = Pipeline([
    ("pca", PCA(n_components=300)),  # Apply PCA for dimensionality reduction
    ("clf", GradientBoostingClassifier())  # Use Gaussian Naive Bayes as the classifier
])

In [ ]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('pca', PCA(n_components=300)),
                ('clf', GradientBoostingClassifier())])

In [ ]:
# Predict on the testing data
ypred = pipeline.predict(X_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, ypred)
print("Accuracy:", accuracy)

Accuracy: 0.9799382716049383


In [ ]:
from sklearn.metrics import classification_report
print('Test Classification Report \n \n',classification_report(y_test,ypred))

Test Classification Report 
 
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       388
           1       0.98      0.97      0.97       260

    accuracy                           0.98       648
   macro avg       0.98      0.98      0.98       648
weighted avg       0.98      0.98      0.98       648



In [ ]:
import cv2
import numpy as np

# Load and preprocess the unseen image
unseen_image_path = "/content/drive/MyDrive/covid.jpeg"
unseen_image = cv2.imread(unseen_image_path)
preprocessed_image = preprocess_images(unseen_image)

# Reshape the preprocessed image to match the input shape of the pipeline model
preprocessed_image = preprocessed_image.reshape(1, -1)

# Make predictions on the preprocessed image
prediction = pipeline.predict(preprocessed_image)

if prediction[0] == 0:
    print("The X-ray image is predicted as COVID-free.")
else:
    print("The X-ray image is predicted as COVID-19 infected.")

The X-ray image is predicted as COVID-19 infected.


In [ ]:
import pickle
#save model
pickle.dump(pipeline, open("mymodel.pkl", "wb"))

In [ ]:
#load model
model = pickle.load(open("mymodel.pkl", "rb"))

In [ ]:
model.predict(preprocessed_image)

array([1])